In [1]:
from utils import *

In [2]:
feature_labels = [
    'number of sent emails',
    'statistics of sent emails\' size',
    'statistics of sent emails\' size',
    'statistics of sent emails\' size',
    'statistics of sent emails\' size',
    'statistics of number of recipients',
    'statistics of number of recipients',
    'statistics of number of recipients',
    'statistics of number of recipients',
    'number of unique recipients',
    'proportion of emails sent to telstra',
    'proportion of emails sent to outsie telstra',
    'number of received emails',
    'statistics of received emails\' size',
    'statistics of received emails\' size',
    'statistics of received emails\' size',
    'statistics of received emails\' size',
    'number of unique senders of received emails',
    'proportion of emails received from telstra',
    'proportion of emails received from outside telstra']

In [3]:
data = pd.read_pickle('parsed_data.pkl')
users = list(data['sender_data'].keys())

BETA = 0.05
TAU = 0.9

In [6]:
# date windows
window = 4
date_windows = []
n_windows = len(SPE) // (2*window) + 1
start = 0
for i in range(n_windows):
    end = start + 2*window
    date_windows.append((start, end))
    start = end

In [5]:
n_anomalies = []
for user in tqdm(users):
    user_send_data = data['sender_data'][user]
    user_receive_data = data['recipient_data'][user]
    features = get_user_features(user, user_send_data, user_receive_data)
    X = scale(features)
    SPE, Q = train_model(X, BETA, TAU)
    n_anomalies.append(count_anomalies(SPE, Q))

100%|██████████| 1683/1683 [00:30<00:00, 54.97it/s]


In [9]:
n_anomalies = np.array(n_anomalies)
user_idxs_to_report = np.where((n_anomalies > 3).sum(1) != 0)[0]

In [10]:
for user_idx in user_idxs_to_report:
    
    user = users[user_idx]

    user_send_data = data['sender_data'][user]
    user_receive_data = data['recipient_data'][user]
    features = get_user_features(user, user_send_data, user_receive_data)
    X = scale(features)
    SPE, Q = train_model(X, BETA, TAU)

    date_window = date_windows[np.argmax(count_anomalies(SPE, Q))]
    day_idxs = np.where(SPE>Q)[0]
    idxs = (day_idxs>=date_window[0]) & (day_idxs<date_window[1])
    days_to_report = day_idxs[idxs]

    print(user)
    for dd in days_to_report:
        print('Anomaly on {} due to {}'.format(
            pd.to_datetime('2017-07-{}'.format(dd+1)).date(),
            feature_labels[X[dd].argmax()]
            )
        )
    print()    

Sarah.Rediger@team.telstra.com
Anomaly on 2017-07-20 due to statistics of received emails' size
Anomaly on 2017-07-21 due to statistics of sent emails' size
Anomaly on 2017-07-22 due to statistics of received emails' size
Anomaly on 2017-07-24 due to statistics of sent emails' size

Nick.Martin@team.telstra.com
Anomaly on 2017-07-17 due to statistics of sent emails' size
Anomaly on 2017-07-19 due to proportion of emails sent to telstra
Anomaly on 2017-07-21 due to statistics of number of recipients
Anomaly on 2017-07-24 due to statistics of received emails' size

Sakshi.Rathore@team.telstra.com
Anomaly on 2017-07-03 due to statistics of sent emails' size
Anomaly on 2017-07-04 due to statistics of sent emails' size
Anomaly on 2017-07-05 due to statistics of sent emails' size
Anomaly on 2017-07-06 due to statistics of received emails' size

Rajashekar.Reddy@team.telstra.com
Anomaly on 2017-07-18 due to proportion of emails sent to telstra
Anomaly on 2017-07-19 due to statistics of receiv